## Прогнозирование стоимости автомобиля по характеристикам


In [2]:
# %pip install catboost
# %pip install sklearn
# %pip install tqdm
# %pip install phik
# %pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
# %pip install seaborn
# %pip install pandas-profiling[notebook]
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [12]:
import xgboost as xg
print(xgboost.__version__)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
#                               RandomForestClassifier, ExtraTreesClassifier)
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier

from sklearn.base import clone

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

import numpy as np  
import pandas as pd 
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
# from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
# %pip freeze > requirements.txt


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))


VERSION = 2

DIR_TEST = './input/sf-dst-car-price-prediction/'
VAL_SIZE = 0.20   # 20%

test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')
train_my = pd.read_csv('dftrain.csv')  # мой датасет для обучения модели

1.5.2
Python       : 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Numpy        : 1.20.3


In [ ]:
# %pip uninstall pandas-profiling
# %pip install pandas-profiling[notebook,html]
# %pip install ruamel-yaml

# использование профилировщика пандас для изучения тестового датасета


# from pandas_profiling import ProfileReport
# profile = ProfileReport(test, title="Pandas Profiling Report")
# profile.to_file("your_report.html")
# profile.to_file("your_trainreport.html")


# разведывательный анализ проведен по результатам работы профилировщика
# на болшее времени не осталось
# исключены ненужные признаки Таможня, Состояние,priceCurrency,image,
#  а так же те, из которых на данном этапе не получится извлечь пользы,
# такие как из котор car_url,description.

# работа со словарями 'complectation_dict','equipment_dict','model_info','super_gen' тоже не проведена
# признак Владение пришлось выбросить на этапе сбора тадасета, т.к. предполагалось времязатратным 
# найти нужные данные, и сформировать аналог записям, которые есть в test.csv (месяцы леты и годы в падежах)
# большая часть времени ушла на сбор тренировочного датасета

## Data Preprocessing

In [13]:
# из-за нехватки времени некоторые признаки в тренировочной базе являются неполными
# адаптировать тестовый датасет, чтобы там не было отсутствующих значений для  таких признаков:
# не все цвета были сопоставлены с номерами, поэтому часть наименее употребимых цветов заменена на'другой цвет'
mycolors = ['чёрный', 'белый', 'серый', 'синий', 'серебристый', 'красный']
test['color'] = test['color'].apply(
    lambda x: x if (x in mycolors) else 'другой цвет')
# моделей с на газе или электричестве тоже в собранном датасете не оказалось
test['fuelType'] = test['fuelType'].apply(lambda x: x if (
    x in ['бензин', 'дизель', 'гибрид']) else 'другое топливо')

# количество дверей у меня набралось числом float - перевести в int
train_my['numberOfDoors'] = train_my['numberOfDoors'].apply(lambda x: int(x))

# test.csv собран 20 октября 2020г 77,9241 RUB за 1 USD 91,3115 RUB за 1 EUR
# мой датасет собран 17 января 2022г 75,7668 RUB за 1 USD 86,8894 RUB за 1 EUR
# соотношение рубля, доллара и евро, судя по цене автомобиля  84.07руб/евро и 74.3руб/USD при курсе на 17 января 2022
# (значит продажа в магазинах в пересчете на валюту чуть по меньшему курсу, чем центробанк)

In [14]:
# попытка учесть изменение курса валюты
from datetime import datetime


def parseunix(x):
    ts = int(x)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

df_ = test.loc[:, ['parsing_unixtime']]
df_['unixtime'] = df_['parsing_unixtime'].apply(parseunix)

df_.sort_values(by='unixtime', ascending=True)
df_.iloc[:2]
# test.csv собран 20 октября 2020г 77,9241 RUB за 1 USD 91,3115 RUB за 1 EUR
# мой датасет собран 17 января 2022г 75,7668 RUB за 1 USD 86,8894 RUB за 1 EUR


,parsing_unixtime,unixtime
0,1603226273,2020-10-20
1,1603226277,2020-10-20


In [15]:
# как отличается курс евро и доллара центробанка от магазинов, если вычислять по стоимости автомобиля в рублях и разных валютах
# train_my['priceadaptEU']=train_my.apply(lambda x: x.price/x.priceEUR,axis=1)
# train_my['priceadaptUS']=train_my.apply(lambda x: x.price/x.priceUSD,axis=1)

now = (86.8894/84.07, 75.7668/74.3)
# если соотношение курса для центробанка и магазинов было такое же, как сейчас,
# то такой был курс валюты в магазинах при пересчете из валюты в рубли
ago = (91.3115/now[0], 77.9241/now[1])
print(ago)

(88.34861105036977, 76.41553596034147)


In [16]:
# адаптировать цену в рублях пересчетом из валюты
train_my['priceadapt'] = train_my.apply(lambda x: np.mean(
    [ago[0]*x.priceEUR, ago[1]*x.priceUSD]), axis=1)
train_my[['price', 'priceEUR', 'priceUSD', 'priceadapt']].sample(5)
# priceadapt - адаптированная цена пересчетом курса валют,
# однако на кагле рейтинг не изменился ни на копейку для двух результатов - с пересчетом цены и без пересчета

,price,priceEUR,priceUSD,priceadapt
13444,1520000.0,18081.0,20459.0,1.580408e+06
15290,340000.0,4044.0,4577.0,3.535178e+05
10898,360000.0,4282.0,4845.0,3.742710e+05
11617,1870000.0,22243.0,25171.0,1.944297e+06
16379,2270000.0,27001.0,30555.0,2.360189e+06


In [17]:
# с этими признаками я не успеваю поработать
# 'complectation_dict','equipment_dict','model_info','super_gen',
# придется их пока выкинуть из базы


# оставшиеся признаки, по которым буду предсказывать цену
columns_my = ['bodyType', 'brand', 'color',
              'engineDisplacement', 'enginePower',
              'fuelType',  'mileage', 'modelDate',  'model_name',
              'name', 'numberOfDoors',
              'productionDate', 'sell_id',  'vehicleConfiguration',
              'vehicleTransmission', 'vendor', 'Владельцы',  'ПТС',
              'Привод', 'Руль']


# это категорийные признаки
cat_features_ids = ['bodyType', 'brand', 'color',
                    'engineDisplacement', 'enginePower',
                    'model_name', 'name', 'fuelType',
                    'numberOfDoors',
                    'vehicleConfiguration',
                    'vehicleTransmission', 'vendor', 'Владельцы',  'ПТС',
                    'Привод', 'Руль']

# обновить признаки категорий в соответствии с выбранными столбцами из датасета
# (при итеративном отключении и включении разных столбцов в тренировочные признаки при попытке выяснить, какой столбец рушит моментально catboost)
cat_features_ids = list(set(columns_my).intersection(set(cat_features_ids)))

In [37]:
# выбрать столбцы для передачи в регрессоры
df_train_my = train_my[columns_my]
y = train_my['price']
df_test = test[columns_my]

df_train_my['sample'] = 1  # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
data_my = df_test.append(df_train_my, sort=False).reset_index(
    drop=True)  # объединяем

C:\Users\User\AppData\Local\Temp/ipykernel_17804/1121124924.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_my['sample'] = 1  # помечаем где у нас трейн
C:\Users\User\AppData\Local\Temp/ipykernel_17804/1121124924.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['sample'] = 0  # помечаем где у нас тест


# Model 1: Перепробование разных регрессоров, ансамблей и стекинга


In [19]:
# класс -обертка для управления регрессорами
class AlgRegressor:
    # словарь регрессоров по короткому имени;
    # везде к именам добавляется нижнее подчеркивание для данных, обработанных StandardScaler или при их использовании
    # копия каждого регрессора для масштабированных данных хранится по такому же имени с нижним подчеркиванием
    algs = {}
    # гиперпараметры только что выполненного поиска  и последнего переданного регрессора
    lastbestparams = None
    rf_random = None  # поисковик гиперпараметров

    def __init__(self, data_my, y, cat_features=None) -> None:
        self.y = y
        if cat_features is None:
            pass
        else:  # перевести признаки из категорийных столбцов в числа
            self.cat_features = cat_features
            for colum in cat_features:
                data_my[colum] = data_my[colum].astype('category').cat.codes
        scaler = StandardScaler()
        sample = set(data_my.columns).difference('sample')
        data_my_ = pd.DataFrame(
            data=scaler.fit_transform(data_my), columns=sample)
        data_my_['sample'] = data_my['sample']
        # приготовить масштабированные данные и
        self.X_ = data_my_.query('sample == 1').drop(['sample'], axis=1)
        self.X_sub_ = data_my_.query('sample == 0').drop(['sample'], axis=1)
        # и неизменные
        self.X = data_my.query('sample == 1').drop(['sample'], axis=1)
        self.X_sub = data_my.query('sample == 0').drop(['sample'], axis=1)
        # приготовить тренировочный и тестовый наборы для масштабированныхъ и нет признаков
        self.X_train_, self.X_test_, self.y_train_, self.y_test_ = train_test_split(
            self.X_, self.y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
        super().__init__()

    def addalg(self, algname, alg):  # добавить в словарь очередной регрессор
        self.algs[algname] = alg
        # продублировать алгоритм для масштабироваанных признаков
        self.algs[algname+"_"] = clone(alg)
        print(algname)

    # вызвать fit и predict для регрессора в словаре
    def fitpredict(self, algname, isScaler=False, isX_sub=False):

        # isScaler-работать с масштабированными признаками
        # isX_sub-делать предикт для рабочего набора из test.csv, иначе - сплитованный набор базы из авто.ру со всеми ценами
        if isScaler:
            alg = self.algs[algname]
        else:
            alg = self.algs[algname+"_"]
        if isX_sub:
            if isScaler:
                return alg.predict(self.X_sub_)
            else:
                return alg.predict(self.X_sub)
        else:
            if isScaler:
                alg.fit(self.X_train_, self.y_train_)
                return alg.predict(self.X_test_)
            else:
                alg.fit(self.X_train, self.y_train)
                return alg.predict(self.X_test)

    def setp(self, algname, **params):  # установить гиперпараметры для р егрессора в словаре
        self.algs[algname].set_params(**params)
        for qq in params:
            print(str(self.algs[algname].get_params()[qq]), end=" ")
        print("\n")

    def fullname_(self, algname):  # вывести имя регрессора, соответствующее короткому имени
        return str(alg.algs[algname]).split(".")[-1].rstrip("'>")

    # запустил только 1 раз для RandomForestregressor - работало несколько часов, закончило где-то ночью неизвестно во сколько
    def SearchCV(self, algname, isScaler=False, rf_random=None):  # подбор гиперпараметров
        if isScaler:
            alg = self.algs[algname]
        else:
            alg = self.algs[algname+"_"]

        if (alg is None):
            raise Exception('не инициализирован регрессор')
        if rf_random is None:
            n_estimators = [int(x) for x in np.linspace(
                start=200, stop=2000, num=10)]
            max_features = ['auto', 'sqrt']
            max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
            max_depth.append(None)
            min_samples_split = [2, 5, 10]
            min_samples_leaf = [1, 2, 4]
            bootstrap = [True, False]
            random_grid = {'n_estimators': n_estimators,
                           'max_features': max_features,
                           'max_depth': max_depth,
                           'min_samples_split': min_samples_split,
                           'min_samples_leaf': min_samples_leaf,
                           'bootstrap': bootstrap}
            self.searchCV = RandomizedSearchCV(estimator=alg, param_distributions=random_grid, n_iter=100,
                                               cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)
        else:
            self.searchCV = rf_random

        if isScaler:
            self.searchCV.fit(self.X_train, self.y_train)
        else:
            self.searchCV.fit(self.X_train_, self.y_train_)
        self.lastbestparams = rf_random.best_params_
        alg.set_params(**self.lastbestparams)

    # функции для стекинга
    # здесь передаются ndarrays , а не dataframe
    def compute_meta_feature(self, clf, X_train, X_test, y_train, cv):
        """
        Computes meta-features using the regressor.
        :arg clf: scikit-learn regressor
        :args X_train, y_train: training set
        :arg X_test: testing set
        :arg cv: cross-validation folding
        """
        X_meta_train = np.zeros_like(y_train, dtype=np.float32)
        for train_fold_index, predict_fold_index in cv.split(X_train):
            X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
            y_fold_train = y_train[train_fold_index]

            folded_clf = clone(clf)
            folded_clf.fit(X_fold_train, y_fold_train)
            X_meta_train[predict_fold_index] = folded_clf.predict(
                X_fold_predict)

        meta_clf = clone(clf)
        meta_clf.fit(X_train, y_train)

        X_meta_test = meta_clf.predict(X_test)

        return X_meta_train, X_meta_test

    def generate_meta_features(self, nregressors,  cv, isScaler=False, isX_sub=False):
        """
        Generates metafeatures using a list of regressors.
        :arg regressors: list of scikit-learn regressors
        :args X_train, y_train: training set
        :arg X_test: testing set
        :arg cv: cross-validation folding
        """
        if isScaler:  # копия каждого регрессора для масштабированных данных хранится по имени с нижним подчеркиванием
            regressors = [alg.algs[i+"_"] for i in nregressors]
            if isX_sub:  # для рабочего набора stacked_features_train взять в качестве тренировочной всю имеющуюся базу, вытянутую из авто.ру
                # а для тестового -stacked_features_test - данные из базы test
                X_train, X_test, y_train = self.X_.to_numpy(
                ), self.X_sub_.to_numpy(), self.y.to_numpy()
            else:
                X_train, X_test, y_train = self.X_train_.to_numpy(
                ), self.X_test_.to_numpy(), self.y_train_.to_numpy()
        else:
            regressors = [alg.algs[i] for i in nregressors]
            if isX_sub:
                X_train, X_test, y_train = self.X.to_numpy(
                ), self.X_sub.to_numpy(), self.y.to_numpy()
            else:
                X_train, X_test, y_train = self.X_train.to_numpy(
                ), self.X_test.to_numpy(), self.y_train_.to_numpy()
        regressorsnew = []
        # не сбивать осуществленный fit у регрессоров,сделать их копии, т.к. для метапараметров будет другой фит
        for rgs in regressors:
            # гиперпараметры тоже копируются или по умолчанию встают???проверить
            regressorsnew.append(clone(rgs))

        features = [
            self.compute_meta_feature(clf, X_train, X_test, y_train, cv)
            for clf in tqdm(regressorsnew)
        ]
        tr = np.vstack([features_train for features_train,
                       features_test in features]).T
        tst = np.vstack([features_test for features_train,
                        features_test in features]).T
        if isScaler:
            if isX_sub:
                self.stacked_features_train_sub_ = tr
                self.stacked_features_test_sub_ = tst
            else:
                self.stacked_features_train_ = tr
                self.stacked_features_test_ = tst
        else:
            if isX_sub:
                self.stacked_features_train_sub = tr
                self.stacked_features_test_sub = tst
            else:
                self.stacked_features_train = tr
                self.stacked_features_test = tst

        return tr, tst

    # когда приготовлены метапризнаки с помощью generate_meta_features, можно вызывать stekpredict
    # для оценки метрики traintest и для рабочего набора test.csv надо отдельно вызывать generate_meta_features
    def stekpredict(self, algname, isScaler=False, isX_sub=False):
        if isScaler:
            alg = clone(self.algs[algname+"_"])
        else:
            alg = clone(self.algs[algname])

        if isX_sub:
            if isScaler:
                alg.fit(self.stacked_features_train_sub_, self.y)
                return alg.predict(self.stacked_features_test_sub_)
            else:
                alg.fit(self.stacked_features_train_sub, self.y)
                return alg.predict(self.stacked_features_test_sub)
        else:
            if isScaler:
                alg.fit(self.stacked_features_train_, self.y_train_)
                return alg.predict(self.stacked_features_test_)
            else:
                alg.fit(self.stacked_features_train, self.y_train)
                return alg.predict(self.stacked_features_test)

In [43]:
q=np.log(y)[:10]
qq=np.exp(q)
qq

0      220000.0
1      490000.0
2      180000.0
3      480000.0
4      440000.0
5     2520000.0
6     4220000.0
7      440000.0
8    10800000.0
9      630000.0
Name: price, dtype: float64

In [52]:
# создать класс -обертку для работы с регрессорами
alg = AlgRegressor(data_my, np.log(y), cat_features_ids)
# добавление разных регрессоров
alg.addalg('LR', LinearRegression())
alg.addalg('XGB', xg.XGBRegressor())

alg.addalg('DTR', DecisionTreeRegressor(random_state=RANDOM_SEED,
           max_depth=14))  # наилучшая глубина подобрана перебором
alg.addalg('RFR', RandomForestRegressor(random_state=RANDOM_SEED))
alg.addalg('Ada', AdaBoostRegressor(random_state=RANDOM_SEED))
alg.addalg('GBoo', GradientBoostingRegressor(learning_rate=0.1, n_estimators=100,
                                             max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1, max_features=int(len(data_my.columns)/3), random_state=RANDOM_SEED))

LR
XGB
DTR
RFR
Ada
GBoo


In [53]:
# проверка для выбранного алгоритма на немасштабированных и масштабированных признаках
def predictionresult(algname):
    predict = (alg.fitpredict(algname), alg.fitpredict(algname, isScaler=True))
    print("Точность {}/(StandardScaler) по метрике MAPE,%: {:0.2f}/{:0.2f}".format(alg.fullname_(algname),
                                                mean_absolute_percentage_error(
                                                    np.exp(alg.y_test_), np.exp(predict[0]))*100,
                                                mean_absolute_percentage_error(np.exp(alg.y_test), np.exp(predict[1]))*100))

In [54]:
# линейная регрессия хуже остальных алгоритмов предсказывает
predictionresult('LR')
# predictionresult('Ada') #показывает MAPE на порядок хуже остальных -чета не то, не успел выяснить

# predictionresult('GBoo') #из коробки хуже, чем с подобранными гиперпараметрами для RandomForestRegressor
predictionresult('XGB') 

bparams = {'n_estimators': 1200, 'min_samples_split': 10,
           'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20}
alg.setp('GBoo', **bparams)
predictionresult('GBoo')

predictionresult('DTR')  # 22.99% /23.20% при max_depth=14
# точность RandomForestRegressor по MAPE из коробки получилась выше, чем с подбором гиперпараметров
predictionresult('RFR')
# всю ночь выполнялся перебор alg.SearchCV('RFR')
# но лучших результатов не получилось
# Fitting 3 folds for each of 100 candidates, totalling 300 fits
# для RandomForestRegressor подобраны гиперпараметры:
# {'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}
# bparams={'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}
# alg.setp('RFR',**bparams)
# predictionresult('RFR')

Точность LinearRegression()/(StandardScaler) по метрике MAPE,%: 30.39/30.42
Точность 300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=2,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)/(StandardScaler) по метрике MAPE,%: 16.75/16.77
1200 10 4 sqrt 20 

Точность GradientBoostingRegressor(max_depth=20, max_features='sqrt', min_samples_leaf=4,
                          min_samples_split=10, n_estimators=1200,
                          random_state=42, subsample=1)/(StandardScaler) по метрике MAPE,%: 20.87/16.60
Точность DecisionTreeRegressor(max_depth=14, random_state=42)/(StandardScaler) по метрике MAPE,%: 22.85/23.36
Точность RandomForestRegressor(random_state=42)/(StandardScaler) по метрике MAPE,%: 16.50/16.56


In [55]:
cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

# запуск стекинга для 4х регрессоров, делающих метапризнаки
nalgs = ['DTR', 'XGB', 'RFR', 'GBoo']  # ,'RFR','GBoo'
stacked_features_train_, stacked_features_test_ = alg.generate_meta_features(
    nalgs, cv, isScaler=True)
stacked_features_train, stacked_features_test = alg.generate_meta_features(
    nalgs, cv)

stacked_features_train_sub_, stacked_features_test_sub_ = alg.generate_meta_features(
    nalgs, cv, isScaler=True,isX_sub=True)
stacked_features_train_sub, stacked_features_test_sub = alg.generate_meta_features(
    nalgs, cv,isX_sub=True)    

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [56]:
nalgs = ['GBoo','DTR', 'XGB', 'RFR']
nalgs.append('LR')
nalgs

['GBoo', 'DTR', 'XGB', 'RFR', 'LR']

In [58]:
# опробование на метапараметрах каждого из регрессоров,
# из которых эти метапараметры генерировались.

predict=None
predict_sub=None

for Alg in nalgs:
    print(Alg)
    if Alg=='GBoo':
        # а GradientBoostingRegressor заупрямился и потребовал max_features must be in (0, n_features]
        # не помогало ни sqrt, ни auto - пришлось для него сбросить 'max_features': None,
        # - только при этом выдал результат MAPE,%: 19.50/20.15
        alg.addalg('GBoo', GradientBoostingRegressor(random_state=RANDOM_SEED))     
    predict = (alg.stekpredict(Alg), alg.stekpredict(Alg, isScaler=True))
    print("Точность стекинг для {}/(StandardScaler) по метрике MAPE,%: {:0.2f}/{:0.2f}".format(alg.fullname_(Alg),
                            mean_absolute_percentage_error(
                                np.exp(alg.y_test_), np.exp(predict[0]))*100,
                            mean_absolute_percentage_error(np.exp(alg.y_test_), np.exp(predict[1]))*100))



GBoo
GBoo
Точность стекинг для GradientBoostingRegressor(random_state=42)/(StandardScaler) по метрике MAPE,%: 15.83/16.11
DTR
Точность стекинг для DecisionTreeRegressor(max_depth=14, random_state=42)/(StandardScaler) по метрике MAPE,%: 19.85/19.47
XGB
Точность стекинг для 300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=2,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)/(StandardScaler) по метрике MAPE,%: 16.52/16.72
RFR
Точность стекинг для RandomForestRegressor(random_state=42)/(StandardScaler) по метрике MAPE,%: 16.60/16.80
LR
Точность стекинг для LinearRegression()/(StandardScaler) по метрике MAPE,%: 15.66/15.90


In [59]:
Alg='LR'
predict_sub = alg.stekpredict(Alg,isX_sub=True)


In [60]:
sample_submission['price'] = predict_sub
sample_submission.to_csv(f'submission_myds_stek_logy.csv', index=False)


# оказалось лучший результат: 
внешний регрессор, который отрабатывает на метапризнаках -LinearRegression

Точность стекинг для LinearRegression()/(StandardScaler) по метрике MAPE,%: 17.73/19.63

но на kaggle выложу пока катбуст, уже дедлайн наступил. позже пересчитаю через стекинг

# # Model 2 : CatBoost


In [ ]:
# с катбустом много вопросов.
# с одной стороны он сразу дал хороший результат на малом количестве признаков.
# с другой стороны - ядро питон умирает по счету раз, если добавлять некоторые признаки.
# самое непонятное - безобидный признак fuelType, которого всего 4 значения уверенно рушил работу катбуста,
# даже при минимальном наборе
# cat_features_ids=['bodyType', 'brand', 'color','fuelType']
# было перепробовано много комбинаций признаков, самый найденный длинный список
# ['Руль', 'Владельцы', 'color', 'enginePower', 'brand', 'bodyType', 'engineDisplacement', 'vehicleConfiguration', 'model_name', 'numberOfDoors', 'name']
# при добавлении новых признаков - ядро сразу умирает при запуске fit
# а с признаком fuelType ядро умирает при очень коротком списке признаков.

In [31]:
# выбрать подмножество признаков, которое не рушит катбуст
columns_my = ['bodyType', 'brand', 'color', 'Владельцы',
              'engineDisplacement', 'enginePower',
              'mileage',     'modelDate', 'model_name',
              'name', 'numberOfDoors',
              'productionDate',   'vehicleConfiguration', 'Руль'
              ]
columns_my = ['model_name', 'enginePower', 'brand', 'name',
              'vehicleConfiguration', 'engineDisplacement', 'bodyType']
# приготовить категориальные признаки для катбуста
# обрезать полный набор cat_features_ids (определен выше) в соответствии с тем, сколько столбцов я оставил в работу
cat_features_ids = list(set(columns_my).intersection(set(cat_features_ids)))

df_train_my = train_my[columns_my]
df_test = test[columns_my]
y = train_my['priceadapt']

df_train_my['sample'] = 1  # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
data_my = df_test.append(df_train_my, sort=False).reset_index(
    drop=True)  # объединяем

X = data_my.query('sample == 1').drop(['sample'], axis=1)
X_sub = data_my.query('sample == 0').drop(['sample'], axis=1)

C:\Users\User\AppData\Local\Temp/ipykernel_12360/2046344508.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_my['sample'] = 1 # помечаем где у нас трейн
C:\Users\User\AppData\Local\Temp/ipykernel_12360/2046344508.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['sample'] = 0 # помечаем где у нас тест


In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
model = CatBoostRegressor(iterations=5000,
                          random_seed=RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                          )
model.fit(X_train, np.log(y_train),
          cat_features=cat_features_ids,
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=20,
          use_best_model=True,
          #  plot=True
          )

# model.save_model('catboost_single_model_2_baseline.model')

Learning rate set to 0.030553
0:	learn: 0.0504515	test: 0.0507964	best: 0.0507964 (0)	total: 371ms	remaining: 30m 55s
20:	learn: 0.0384732	test: 0.0383104	best: 0.0383104 (20)	total: 2.79s	remaining: 11m
40:	learn: 0.0325580	test: 0.0318106	best: 0.0318106 (40)	total: 5.11s	remaining: 10m 17s
60:	learn: 0.0295638	test: 0.0283222	best: 0.0283222 (60)	total: 7.65s	remaining: 10m 19s
80:	learn: 0.0281920	test: 0.0267296	best: 0.0267296 (80)	total: 9.96s	remaining: 10m 5s
100:	learn: 0.0273921	test: 0.0257961	best: 0.0257961 (100)	total: 12.3s	remaining: 9m 55s
120:	learn: 0.0268667	test: 0.0252638	best: 0.0252638 (120)	total: 14.6s	remaining: 9m 48s
140:	learn: 0.0264740	test: 0.0248711	best: 0.0248711 (140)	total: 17.1s	remaining: 9m 48s
160:	learn: 0.0261530	test: 0.0245624	best: 0.0245624 (160)	total: 19.6s	remaining: 9m 47s
180:	learn: 0.0258508	test: 0.0242867	best: 0.0242867 (180)	total: 22.3s	remaining: 9m 54s
200:	learn: 0.0255605	test: 0.0239911	best: 0.0239911 (200)	total: 25s	r

In [33]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))
print(
    f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 28.86%


# Submission

In [34]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_myds_v{VERSION}.csv', index=False)
sample_submission.head(10)
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   sell_id  34686 non-null  int64  
 1   price    34686 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 542.1 KB


# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing